<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc" style="margin-top: 1em;"><ul class="toc-item"><li><span><a href="#导入相关的包" data-toc-modified-id="导入相关的包-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>导入相关的包</a></span></li><li><span><a href="#导入相关数据" data-toc-modified-id="导入相关数据-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>导入相关数据</a></span></li><li><span><a href="#形成训练数据" data-toc-modified-id="形成训练数据-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>形成训练数据</a></span></li><li><span><a href="#训练模型" data-toc-modified-id="训练模型-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>训练模型</a></span></li></ul></div>

# 导入相关的包

In [47]:
import pandas as pd
import numpy as np
import xgboost as xgb
import time
import warnings
from sklearn.model_selection import train_test_split
warnings.filterwarnings("ignore")

# 导入相关数据

In [6]:
%%time
trains = pd.read_csv('../raw_data/d_train.csv',encoding="gbk")
tests = pd.read_csv("../raw_data/d_test_A.csv",encoding="gbk")
trains.drop(trains[trains["年龄"] >= 84].index,inplace=True)
fea_train = pd.read_csv("../raw_data/fea_train.csv")
fea_test = pd.read_csv("../raw_data/fea_test.csv")
fea_train1 = pd.read_csv("../raw_data/fea_train_1.csv")
fea_test1 = pd.read_csv("../raw_data/fea_test_1.csv")
trains = pd.merge(trains, fea_train, how="left",on="id")
trains = pd.merge(trains, fea_train1, how="left",on="id")
tests = pd.merge(tests, fea_test, how="left",on="id")
tests = pd.merge(tests, fea_test1, how="left",on="id")
trains["血糖"] = trains["血糖"].apply(lambda x: 1 if x < 4.5 else 0)
trains["血糖"] = trains["血糖"].astype(np.int32)

Wall time: 238 ms


In [7]:
sum(trains["血糖"])

307

# 形成训练数据

In [42]:
%%time
def make_feat(train,test):
    train_id = train.id.values.copy()
    test_id = test.id.values.copy()
    data = pd.concat([train,test])
    data['性别'] = data['性别'].map({'男': 1,'女': 0})
    # data["性别"] = data['性别'].astype(int)
    # data['date'] = (pd.to_datetime(data['date']) - parse('2017-10-09')).dt.days
    # data['体检日期'] = (pd.to_datetime(data['体检日期']) - parse('2017-10-09')).dt.days
    data.drop("体检日期",axis = 1,inplace= True)
    # data.fillna(value = -1, inplace = True)
    train_feat = data[data.id.isin(train_id)]
    test_feat = data[data.id.isin(test_id)]
    return train_feat,test_feat
train, test = make_feat(trains, tests)
predictors = [f for f in list(train.columns) if f not in ["血糖","blood_sugar","id","blood_sugar_log","体检日期"]]
# X_train, X_test, y_train,y_test = train_test_split(train[predictors], train["血糖"],test_size=0.1,random_state=42)
xgb_train = xgb.DMatrix(train[predictors], label=train["血糖"])
# xgb_eval = xgb.DMatrix(X_test, label=y_test)
xgb_test = xgb.DMatrix(test[predictors])

Wall time: 96.6 ms


In [9]:
train.shape
test.shape
test.head()
train.head()
(len(train["血糖"]) - sum(train["血糖"])) / sum(train["血糖"])

(5622, 93)

(1000, 93)

,*r-谷氨酰基转换酶,*丙氨酸氨基转换酶,*天门冬氨酸氨基转换酶,*总蛋白,*球蛋白,*碱性磷酸酶,ATSm,ATSs,feature_0_feature_1,feature_0_is_normal,...,红细胞平均血红蛋白量,红细胞计数,肌酐,血小板体积分布宽度,血小板平均体积,血小板比积,血小板计数,血糖,血红蛋白,高密度脂蛋白胆固醇
0,34.36,26.69,23.85,82.75,36.72,116.08,4.7700,19.0800,50.54,1,...,30.6,4.94,65.87,12.8,10.8,0.26,241.0,NaN,151.0,1.44
1,111.43,34.98,29.75,71.90,27.81,90.07,5.9500,23.8000,64.73,1,...,29.3,5.84,89.99,14.6,11.5,0.28,242.0,NaN,171.0,1.22
2,NaN,NaN,NaN,NaN,NaN,NaN,5.6388,22.5552,61.60,1,...,29.9,5.72,73.29,9.9,8.9,0.35,398.0,NaN,171.0,NaN
3,23.41,16.63,17.98,78.16,32.72,95.95,3.5960,14.3840,34.61,1,...,29.9,4.61,70.82,13.9,11.6,0.29,247.0,NaN,138.0,1.38
4,15.70,19.80,19.12,80.76,33.86,76.97,3.8240,15.2960,38.92,1,...,31.8,4.62,73.91,11.9,10.4,0.35,335.0,NaN,147.0,2.25


,*r-谷氨酰基转换酶,*丙氨酸氨基转换酶,*天门冬氨酸氨基转换酶,*总蛋白,*球蛋白,*碱性磷酸酶,ATSm,ATSs,feature_0_feature_1,feature_0_is_normal,...,红细胞平均血红蛋白量,红细胞计数,肌酐,血小板体积分布宽度,血小板平均体积,血小板比积,血小板计数,血糖,血红蛋白,高密度脂蛋白胆固醇
0,20.23,23.10,24.96,76.88,27.28,99.59,4.992,19.968,48.06,1,...,31.9,5.21,77.25,17.4,9.9,0.164,166.0,0.0,166.1,1.37
1,79.00,36.25,24.57,79.43,31.67,67.21,4.914,19.656,60.82,1,...,29.9,5.21,87.12,10.3,9.2,0.260,277.0,0.0,156.0,0.93
2,38.17,15.23,20.82,86.23,38.23,63.69,4.164,16.656,36.05,1,...,31.3,4.76,78.19,16.6,8.3,0.199,241.0,0.0,148.8,1.64
3,20.22,10.59,14.99,70.98,26.96,74.08,2.998,11.992,25.58,1,...,31.9,4.29,61.46,10.8,10.3,0.260,252.0,1.0,137.0,1.43
4,22.72,14.78,20.07,78.05,36.22,75.79,4.014,16.056,34.85,1,...,20.6,5.15,NaN,14.0,11.1,0.350,316.0,0.0,106.0,1.27


17.312703583061889

# 训练模型

In [25]:
params={'booster':'gbtree',
    'objective': 'binary:logistic',
    'scale_pos_weight':float(len(train["血糖"]) - sum(train["血糖"])) / sum(train["血糖"]),
    'eval_metric': 'auc',
    "subsample":0.65,
    "colsample_bytree":0.8,
    "lambda":5,
    "seed":1024,
    "silent":1,
    "verbose":0,
    "max_depth":7,
    "alpha":0.01,
    "gamma":0.6
    }

In [38]:
%%time
watchlist  = [(xgb_train,'train')]
#通过cv找最佳的nround
cv_log = xgb.cv(params,xgb_train,num_boost_round=25000,nfold=5,metrics='auc',early_stopping_rounds=50,seed=1024)

Wall time: 19.6 s


In [39]:
cv_log

,test-auc-mean,test-auc-std,train-auc-mean,train-auc-std
0,0.615297,0.025862,0.795399,0.006983
1,0.618756,0.020102,0.867670,0.013116
2,0.638427,0.025884,0.901220,0.007781
3,0.642705,0.035393,0.926528,0.005198
4,0.644796,0.047826,0.942466,0.008811
5,0.642739,0.045066,0.957778,0.008484
6,0.640418,0.040436,0.967148,0.008576
7,0.640693,0.031189,0.974289,0.006569
8,0.647643,0.027583,0.979952,0.004054


In [40]:
%%time
bst_auc= cv_log['test-auc-mean'].max()
cv_log['nb'] = cv_log.index
cv_log.index = cv_log['test-auc-mean']
bst_nb = cv_log.nb.to_dict()[bst_auc]
#train
watchlist  = [(xgb_train,'train')]
model = xgb.train(params,xgb_train,num_boost_round=bst_nb+50,evals=watchlist)

[0]	train-auc:0.806118
[1]	train-auc:0.862662
[2]	train-auc:0.895197
[3]	train-auc:0.918388
[4]	train-auc:0.929593
[5]	train-auc:0.942765
[6]	train-auc:0.95175
[7]	train-auc:0.960436
[8]	train-auc:0.971075
[9]	train-auc:0.973919
[10]	train-auc:0.979703
[11]	train-auc:0.981893
[12]	train-auc:0.983154
[13]	train-auc:0.98628
[14]	train-auc:0.988712
[15]	train-auc:0.989892
[16]	train-auc:0.992174
[17]	train-auc:0.994212
[18]	train-auc:0.995548
[19]	train-auc:0.996664
[20]	train-auc:0.997941
[21]	train-auc:0.99844
[22]	train-auc:0.998747
[23]	train-auc:0.998954
[24]	train-auc:0.999239
[25]	train-auc:0.999295
[26]	train-auc:0.999489
[27]	train-auc:0.999678
[28]	train-auc:0.999755
[29]	train-auc:0.999798
[30]	train-auc:0.999805
[31]	train-auc:0.999819
[32]	train-auc:0.999819
[33]	train-auc:0.999812
[34]	train-auc:0.99978
[35]	train-auc:0.999882
[36]	train-auc:0.999933
[37]	train-auc:0.99993
[38]	train-auc:0.99996
[39]	train-auc:0.999979
[40]	train-auc:0.999991
[41]	train-auc:0.999997
[42]	tra

In [49]:
test_pred = model.predict(xgb_test)

In [52]:

test_result = test
test_result["lt45_prob"] = test_pred
# test_result.to_csv("gt98_prob.csv",index=None,encoding='utf-8')
test_result[["id","lt45_prob"]].to_csv("../classifier/lt45.csv",index=False)
# 

In [24]:
test_result["lt45_prob"].describe()

count    1000.000000
mean        0.478616
std         0.025524
min         0.435223
25%         0.458443
50%         0.473165
75%         0.498612
max         0.544222
Name: lt45_prob, dtype: float64